# Permutation Flow-Shop Scheduling Problem

This is a variant of the Flot-shop scheduling problem (FSSP) in which the sequence of jobs is the same in every machine.

$$
 \begin{align}
     \text{min} \quad & C_{\text{max}} \\
     \text{s.t.} \quad & x_{m-1, j} + p_{m, j} \leq x_{m, j}
         & \forall ~ j \in J; m \in (2, ..., |M|)\\
     & x_{m, j} + p_{m, j} \leq x_{m, k} \lor x_{m, k} + p_{m, k} \leq x_{m, j}
         & \forall ~ j \in J; k \in J, j \neq k\\
     & x_{|M|, j} + p_{|M|, j} \leq C_{\text{max}}
         & \forall ~ j \in J\\
     & x_{m, j} \geq 0 & \forall ~ j \in J; m \in M\\
     & z_{j, k} \in \{0, 1\} & \forall ~ j \in J; k \in J, j \neq k\\
 \end{align}
 $$

 You can compare this implementation to MILP solvers at the [end of the notebook](#bonus---milp-model).

In [1]:
import json
import time

import pandas as pd

from bnbprob.pfssp import CallbackBnB, PermFlowShop
from bnbprob.pfssp.cython.problem import PermFlowShop2MHalf
from bnbpy import configure_logfile

In [2]:
configure_logfile("pfssp-experiments-2mhalf.log", mode="w")

In [3]:
import gc

gc.disable()

In [ ]:
solutions = {}
# problems = [f'ta{str(i).zfill(3)}' for i in range(41, 51, 1)]
problems = (
    [f'ta{str(i).zfill(3)}' for i in range(11, 21, 1)]
    + [f'ta{str(i).zfill(3)}' for i in range(41, 51, 1)]
)

bnbs: dict[str, CallbackBnB] = {}

for prob in problems:
    with open(
        f'./../data/flow-shop/{prob}.json', mode='r', encoding='utf8'
    ) as f:
        p = json.load(f)
        problem = PermFlowShop2MHalf.from_p(p, constructive='neh')
        bnb = CallbackBnB(
            eval_node='in',
            rtol=0.0001,
            heur_factor=100,
            save_tree=False,
            queue_mode='dfs',  # 'dfs' or 'cycle'
        )
        bnb.log_row(prob)
        gc.collect()
        time.sleep(0.1)

        start = time.time()
        sol = bnb.solve(problem, maxiter=100_000_000, timelimit=3600 * 4)
        duration = time.time() - start
        solutions[prob] = (sol, duration)

        print(f"{prob}: {sol} in {duration:.2f}s ({bnb.explored} nodes)")

        bnbs[prob] = bnb
        time.sleep(0.1)

ta011: Status: OPTIMAL | Cost: 1582.0 | LB: 1582.0 in 10.37s (192410 nodes)
ta012: Status: OPTIMAL | Cost: 1659.0 | LB: 1659.0 in 14.68s (257273 nodes)
ta013: Status: OPTIMAL | Cost: 1496.0 | LB: 1496.0 in 28.50s (577969 nodes)
ta014: Status: OPTIMAL | Cost: 1377.0 | LB: 1377.0 in 2.83s (61276 nodes)
ta015: Status: OPTIMAL | Cost: 1419.0 | LB: 1419.0 in 5.32s (104772 nodes)
ta016: Status: OPTIMAL | Cost: 1397.0 | LB: 1397.0 in 7.19s (143616 nodes)
ta017: Status: OPTIMAL | Cost: 1484.0 | LB: 1484.0 in 609.86s (9866201 nodes)
ta018: Status: OPTIMAL | Cost: 1538.0 | LB: 1538.0 in 15.82s (278478 nodes)
ta019: Status: OPTIMAL | Cost: 1593.0 | LB: 1593.0 in 1.76s (39489 nodes)
ta020: Status: OPTIMAL | Cost: 1591.0 | LB: 1591.0 in 31.19s (510811 nodes)
ta041: Status: OPTIMAL | Cost: 2991.0 | LB: 2991.0 in 107.30s (431019 nodes)
ta042: Status: OPTIMAL | Cost: 2867.0 | LB: 2867.0 in 9920.30s (50129607 nodes)
ta043: Status: OPTIMAL | Cost: 2839.0 | LB: 2839.0 in 10110.61s (70852293 nodes)
ta044:

In [ ]:
# Export to dataframe
problems_ = []
ub = []
lb = []
gap = []
time_ = []
n_nodes = []

for key, val in solutions.items():
    sol, t = val
    problems_.append(key)
    ub.append(sol.cost)
    lb.append(sol.lb)
    gap.append(abs(sol.cost - sol.lb) / sol.cost)
    time_.append(t)
    n_nodes.append(bnbs[key].explored)


df = pd.DataFrame({
    "problem": problems_,
    "ub": ub,
    "lb": lb,
    "gap": gap,
    "nodes": n_nodes,
    "time": time_
})

df.to_excel("bench-pfssp-2mhalf-dfs.xlsx")

ValueError: too many values to unpack (expected 2)